In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import fire
import os
from dotenv import load_dotenv

In [17]:
vectorstore_path = "./vectorstores/test_vectorstore"
embeddings = OpenAIEmbeddings()

if os.path.exists(vectorstore_path):
    db = FAISS.load_local(
        vectorstore_path, embeddings, allow_dangerous_deserialization=True
    )
else:
    loader = CSVLoader(file_path="../data/authors_with_papers.csv", encoding="utf-8")
    data = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    documents = text_splitter.split_documents(data)

    db = FAISS.from_documents(documents, embeddings)
    db.save_local(vectorstore_path)


retriever = db.as_retriever(search_kwargs={"k": 5})

PROMPT_TEMPLATE = """
You are an expert in providing information about thesis supervisors at Politechnika Wrocławska.
When given a user question about the most suitable thesis supervisors for their project based on their reasearch papers and intrests, provide 3 supervisor's and their' faculty along with titles of some of their research papers.

User Question:
{question}

Response:
"""

In [18]:
question: str = "Who should I pick as promotor for my engineering thesis 'Train Rescheduling and Track Closure Optimization'"
formatted_prompt = PROMPT_TEMPLATE.format(question=question)

In [19]:
llama3 = Ollama(model='llama3') # 8.0B params

qa_chain = RetrievalQA.from_chain_type(
    llm=llama3,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

print(qa_chain.invoke(formatted_prompt)["result"])

Based on your research papers and interests, I would recommend the following three thesis supervisors at Politechnika Wrocławska who might be a good fit for your project on "Train Rescheduling and Track Closure Optimization":

1. **dr inż. Robert Wójcik**, Faculty of Information and Communication Technology
	* Research Papers: ["The performance evaluation tool for automated prototyping of concurrent cyclic processes", "Design of admissible schedules for AGV systems with constraints: a logic-algebraic approach"]
	* Reasoning: Dr. Wójcik's research focuses on process scheduling, concurrency, and distributed control, which aligns with your project's optimization aspects.
2. **dr inż. Adrianna Filipiak-Kaczmarek**, Faculty of Mechanical Engineering
	* Research Papers: ["Design and implementing possibilities of composite pontoon bridge"]
	* Reasoning: Dr. Filipiak-Kaczmarek's expertise lies in mechanical engineering, which might be relevant to the optimization aspects of your project.
3. **

In [22]:
gemma2 = Ollama(model='gemma2') # 9.2B params

qa_chain = RetrievalQA.from_chain_type(
    llm=gemma2,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

print(qa_chain.invoke(formatted_prompt)["result"])

Based on your topic "Train Rescheduling and Track Closure Optimization,"  I suggest considering these supervisors at Politechnika Wrocławska:

**1. dr inż. Robert Wójcik** 
* **Faculty:** Faculty of Information and Communication Technology 
* **Relevant Research Papers:**
    * "Multimodal processes rescheduling: cyclic steady states space approach" -  This paper deals with optimization and rescheduling in production environments, which overlaps with your train rescheduling theme.
    * "AGVs distributed control subject to imprecise operation times" - This paper explores the challenges of managing automated systems with uncertain operations, relevant to the complexities of track closure optimization.

**2. dr inż. [Supervisor Name -  If you can provide any potential names based on your research or department] **
* **Faculty:** [Faculty Name] 
* **Relevant Research Papers:**  *[Add papers here if you have specific information about potential supervisors]*


Remember to carefully review 

### TODO: Try larger models